In [ ]:
import json

import duckdb
import pandas as pd

### 데이터 전처리 및 정리

In [ ]:
df_res = duckdb.read_csv("./data/2024-07-03 1_19pm.tsv", header=True, delimiter="\t")
df_res = df_res.to_df()
df_res["FILE_NAME"] = df_res["FILE_NAME"].apply(lambda x: x.replace('"', ""))
df_res

In [ ]:
df_origin = duckdb.read_csv(
    ["data/voc-test-sample-2nd.csv", "data/voc-test-sample.csv"],
    header=True,
    delimiter=",",
    quotechar='"',
)
df_origin = df_origin.to_df()
df_origin = df_origin.apply(lambda x: x.str.replace('"', ""))
df_origin

In [ ]:
df = df_origin.merge(df_res, left_on="file_name", right_on="FILE_NAME", how="inner")
df

In [ ]:
results = []

for idx, row in df.iterrows():
    result = json.loads(row["RES"])

    file_name = row["FILE_NAME"]
    original = row["text"]
    model = {
        "asr": "whisper-large-v3",
        "llm": {"model": result["model"], "usage": result["usage"]},
    }

    result = json.loads(json.loads(row["RES"])["choices"][0]["messages"])

    temp = []

    if isinstance(result["title"], list):
        result["title"] = result["title"][0]

    for content in result["contents"]:
        if isinstance(content, list):
            temp.append(content[0])
        else:
            temp.append(content)

    result["contents"] = temp
    temp = []

    for category in result["categories"]:
        if isinstance(category, list):
            temp.append(category[0])
        else:
            temp.append(category)

    result["categories"] = temp
    temp = []

    for tag in result["tags"]:
        if isinstance(tag, list):
            temp.append(tag[0])
        else:
            temp.append(tag)

    result["tags"] = temp
    temp = []

    for category in result["sentiment"]["categories"]:
        if isinstance(category, list):
            temp.append(category[0])
        else:
            temp.append(category)

    result["sentiment"]["categories"] = temp
    temp = []

    if isinstance(result["sentiment"]["description"], list):
        result["sentiment"]["description"] = result["sentiment"]["description"][0]

    result = {"file_name": file_name, "original": original} | result | {"model": model}

    print(result)

    results.append(result)

In [ ]:
datas = {"data": results}

with open("data.json", "w") as f:
    json.dump(datas, f, ensure_ascii=False, indent=4)

### 데이터 분석

In [ ]:
df = duckdb.read_json("data.json")
# df = pd.DataFrame()

df = pd.json_normalize(df["data"].to_df()["data"][0])
df